In [191]:
import requests
import json
import pandas

api_key = 'RGAPI-3f2f63fb-f8b7-4027-9f04-b853613a8455'

def get_match_infos_from_id(matchId, api_key):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_{matchId}'
    header = {'api_key':api_key}
    r = requests.get(url, params=header)
    return r.json()

def get_id_from_url(matchUrl):
    match_details = matchUrl.split('#match-details/')[1]
    return match_details.split('/')[1]

def get_match_infos_from_url(matchUrl, api_key):
    matchId = get_id_from_url(matchUrl)
    return get_match_infos_from_id(matchId, api_key)

specific_match_infos = get_match_infos(specific_matchId, api_key)

In [194]:
get_match_infos_from_url('https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2241100611/200298930?tab=overview', api_key)

{'metadata': {'dataVersion': '2',
  'matchId': 'BR1_2241100611',
  'participants': ['4Tjx5wkk6t-OYhQy8xa0YoM1_qfJrhW2Bu5TvJqWxEZvn5vZkZp5lfS63lazleeIKbMjzYvLDWYYgA',
   'EmKx9nq2rzSIrdIr5kLF4zGj-NPzJjgnsQMyS9Mx3NDmyfeF6c6CMsO4QQPkZ1AiZ4C2CNUIKJsl2Q',
   'bBpLEq0sR1IEcQF87FjgqsyvAIeI1K4IRsKfK34cduPPFxd2y8bA7zmftExYgUGAwh5s95ap1FPzLg',
   'xCKBT0tot0uuxOWgu4XtY5PgV3RmZTEgIioDOyXcgwJTfS5JvU0qr_z4hxWNhWQOcg1r628UClmDPw',
   '10cPWvvnXTFsCb6EuDB294AhgjlOC7rOj8ae1n8T1TangatUPWXAdV19NQm0BY4-swACuLiGrQzSUw',
   'bWt0RgcQs35IAt0LVTr9CEGvdes_Fyb8ETdmMuGaDgaqVmVuztAgTzAgAJqAPx3sjsYcpBuc6MhWxQ',
   'g27W3X-zkOvoOi7rpdKv2ePHSbxGc7udM8SZLu98sI3IFAvb7UxtJVwtdtMVr2laUDnKNLr-QEjJEg',
   'jl5ryr6--QBuzlMkefevt_yuDsObIintF2PwBU5slPM_h-iv6uPjudop0JtfnaGi2cieIlJQjasEVA',
   '3qwJ82RBkGLwUjiupp6pMQLMQ4qMUEM5A4vn8gR3eCKEAXL5SHc_4SggybPCASvG2JZ5MjePieFTKg',
   'S4qAJwbNnvF-luGujDK-_E39AMlqN7I6TwgzQHs696hofcGlMniHYpU88b51uCk2Ljm2kO__6P6tGA']},
 'info': {'gameCreation': 1618549271000,
  'gameDuration': 2170240,